In [2]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, firestore
import requests
import pandas as pd

INFO: pip is looking at multiple versions of google-cloud-storage to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.8/336.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 41.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('/work/staklimjerukagung-firebase-admin.json')  # ganti dengan path ke service account key
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [ ]:
# Referensi Firestore
firestore_db = firestore.client()

In [ ]:
# URL Firebase RTDB untuk mengambil data
url_rtdb = 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/auto_weather_stat/id-03/data.json'

In [ ]:
# Pindahkan data ke Firestore dengan struktur sensor_iklim -> id-01 -> data
collection_ref = firestore_db.collection('sensor_iklim').document('id-01').collection('data')

In [4]:
# Ambil data dari Firebase RTDB
response = requests.get(url_rtdb)
if response.status_code == 200:
    rtdb_data = response.json()

    # Convert RTDB data ke DataFrame pandas
    df = pd.DataFrame.from_dict(rtdb_data, orient='index')
    df['timestamp'] = df.index  # Memindahkan timestamp dari index ke kolom

    print("Data dari RTDB berhasil dibaca:")
    print(df.head())  # Print beberapa baris untuk verifikasi

    for _, row in df.iterrows():
        collection_ref.document(row['timestamp']).set(row.to_dict())  # Simpan setiap baris data

    print("Data berhasil dipindahkan ke Firestore.")
else:
    print("Gagal mengambil data dari RTDB:", response.status_code)

Data dari RTDB berhasil dibaca:
              dew humidity pressure temperature   timestamp  volt
0           22.62    89.33  1011.29       24.49           0  4.06
1702296831  23.29    89.28  1010.72       25.18  1702296831  4.08
1702296891  23.29    89.38  1010.75       25.16  1702296891  4.08
1702296951  23.31    89.58  1010.76       25.14  1702296951  4.08
1702297011  23.35    89.86  1010.82       25.13  1702297011  4.08


RetryError: Deadline of 60.0s exceeded while calling target function, last exception: 429 Quota exceeded.

In [5]:
# Mengambil dokumen dengan urutan timestamp terbaru
query = collection_ref.order_by('timestamp', direction=firestore.Query.DESCENDING).limit(1)
results = query.stream()

# Menampilkan timestamp terakhir
for doc in results:
    latest_data = doc.to_dict()
    print("Timestamp terakhir:", latest_data['timestamp'])

Timestamp terakhir: 1702337099


In [1]:
import firebase_admin
from firebase_admin import credentials, db

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('/work/staklimjerukagung-firebase-admin.json')  # Sesuaikan dengan path file JSON
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'  # Ganti dengan URL Firebase RTDB kamu
})

# Referensi ke Firebase RTDB
ref = db.reference('auto_weather_stat/id-03/data')

# Query untuk mengambil data dengan timestamp terbaru
query = ref.order_by_key().limit_to_last(10)  # Mengambil satu item terakhir berdasarkan timestamp (key)
result = query.get()

# Menampilkan timestamp terakhir
if result:
    for key, value in result.items():
        print("Timestamp terakhir:", key)  # key akan mengandung nilai timestamp
        print("Data terakhir:", value)
else:
    print("Tidak ada data ditemukan.")


Timestamp terakhir: 1728823099
Data terakhir: {'dew': '26.05', 'humidity': '89.25', 'pressure': '1011.27', 'temperature': '27.99', 'timestamp': '1728823099', 'volt': '4.06'}
Timestamp terakhir: 1728823158
Data terakhir: {'dew': '26.07', 'humidity': '89.32', 'pressure': '1011.24', 'temperature': '27.99', 'timestamp': '1728823158', 'volt': '4.06'}
Timestamp terakhir: 1728823219
Data terakhir: {'dew': '26.07', 'humidity': '89.31', 'pressure': '1011.31', 'temperature': '27.99', 'timestamp': '1728823219', 'volt': '4.06'}
Timestamp terakhir: 1728823279
Data terakhir: {'dew': '26.02', 'humidity': '89.23', 'pressure': '1011.25', 'temperature': '27.96', 'timestamp': '1728823279', 'volt': '4.07'}
Timestamp terakhir: 1728823338
Data terakhir: {'dew': '26.04', 'humidity': '89.18', 'pressure': '1011.32', 'temperature': '27.99', 'timestamp': '1728823338', 'volt': '4.07'}
Timestamp terakhir: 1728823398
Data terakhir: {'dew': '26.03', 'humidity': '89.28', 'pressure': '1011.30', 'temperature': '27.96',

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
from tqdm import tqdm  # Import library untuk progress bar

# Inisialisasi Firebase Admin SDK untuk Firebase RTDB sumber
cred_source = credentials.Certificate('/work/staklimjerukagung-firebase-admin.json')  # Ganti dengan path ke file JSON sumber
firebase_admin.initialize_app(cred_source, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'  # Ganti dengan URL Firebase RTDB sumber
}, name='Jerukagung-seismologi')  # Gunakan name untuk inisialisasi multi-app

# Inisialisasi Firebase Admin SDK untuk Firebase RTDB tujuan
cred_dest = credentials.Certificate('/work/database-sensor-iklim-litbang-firebase-admin.json')  # Ganti dengan path ke file JSON tujuan
firebase_admin.initialize_app(cred_dest, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'  # Ganti dengan URL Firebase RTDB tujuan
}, name='Penelitian-sensor-iklim')  # Gunakan name yang berbeda

# Referensi Firebase RTDB sumber dan tujuan
ref_source = db.reference('auto_weather_stat/id-03/data', app=firebase_admin.get_app('Jerukagung-seismologi'))
ref_dest = db.reference('auto_weather_stat/id-03/data', app=firebase_admin.get_app('Penelitian-sensor-iklim'))

# Ambil semua data dari Firebase RTDB sumber
source_data = ref_source.get()

# Cek apakah data ditemukan
if source_data:
    # Ubah data dari RTDB ke DataFrame pandas
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Menampilkan DataFrame hasil (opsional)
    print(df.head())  
    
    # Menggunakan tqdm untuk menampilkan progress bar
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Memindahkan data"):
        # Kirim setiap baris secara terpisah untuk menghindari batasan ukuran
        ref_dest.child(str(index)).set(row.to_dict())  # Menggunakan child untuk menulis setiap data secara individual
    
    print("Data berhasil dipindahkan ke Firebase RTDB tujuan secara bertahap.")
else:
    print("Tidak ada data yang ditemukan di Firebase RTDB sumber.")


              dew humidity pressure temperature   timestamp  volt
0           22.62    89.33  1011.29       24.49           0  4.06
1702296831  23.29    89.28  1010.72       25.18  1702296831  4.08
1702296891  23.29    89.38  1010.75       25.16  1702296891  4.08
1702296951  23.31    89.58  1010.76       25.14  1702296951  4.08
1702297011  23.35    89.86  1010.82       25.13  1702297011  4.08
Memindahkan data:   1%|          | 4273/429742 [17:23<29:05:51,  4.06it/s]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>